In [37]:
import pandas as pd
import numpy as np
import psycopg2
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

%matplotlib inline

In [61]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''
    select * from 
(
select 
distinct a. subject_id,
a.gender,
a.anchor_age,
a.dod,
to_date(REPLACE(a.ecgtime,'2130','2023'),'YYYY-MM-DD') ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_onset,
a.p_end,
a.qrs_onset,
a.qrs_end,
a.t_end,
a.p_axis,
a.qrs_axis,
a.t_axis,
a.target_variable
from 
factihealth.mimic.ecg_model_dataset a
inner join 
factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where date_part(year,b.edadmittime) = 2130
and date_part(year,a.ecgtime) = 2130
and ecgtime between b.edadmittime and b.eddischargetime
order by ecgtime,a.subject_id
)
--where ecg_time between '2023-01-01' AND '2023-01-31'
    ''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)
    

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (171, 17)


In [62]:
def pre_process_ecg(data):
    data.drop(columns=['dod','ecg_time','target_variable'],inplace=True)
    df_encoded = pd.get_dummies(data, columns=['bandwidth','filtering','gender'])
    df_sorted = df_encoded.sort_index(axis=1)
    return df_encoded

In [63]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   subject_id       171 non-null    int64 
 1   gender           171 non-null    object
 2   anchor_age       171 non-null    int64 
 3   dod              53 non-null     object
 4   ecg_time         171 non-null    object
 5   bandwidth        171 non-null    object
 6   filtering        171 non-null    object
 7   rr_interval      171 non-null    int64 
 8   p_onset          171 non-null    int64 
 9   p_end            171 non-null    int64 
 10  qrs_onset        171 non-null    int64 
 11  qrs_end          171 non-null    int64 
 12  t_end            171 non-null    int64 
 13  p_axis           171 non-null    int64 
 14  qrs_axis         171 non-null    int64 
 15  t_axis           171 non-null    int64 
 16  target_variable  171 non-null    object
dtypes: int64(11), object(6)
memory usag

In [65]:
master_data[['subject_id']]

,subject_id
0,13130347
1,12133624
2,13494259
3,15985786
4,15440644
...,...
166,14992632
167,14992632
168,15301471
169,16926575


In [25]:
data = pre_process_ecg(master_data)
data.info()
processed_data=data.drop(columns='subject_id').sort_index(axis=1)
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   subject_id                             171 non-null    int64
 1   anchor_age                             171 non-null    int64
 2   rr_interval                            171 non-null    int64
 3   p_onset                                171 non-null    int64
 4   p_end                                  171 non-null    int64
 5   qrs_onset                              171 non-null    int64
 6   qrs_end                                171 non-null    int64
 7   t_end                                  171 non-null    int64
 8   p_axis                                 171 non-null    int64
 9   qrs_axis                               171 non-null    int64
 10  t_axis                                 171 non-null    int64
 11  bandwidth_0.0005-150 Hz         

In [35]:
import pickle
with open('icu_patient_ecg_prediction.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

ecg_output=pd.DataFrame(loaded_model.predict(processed_data)).reset_index()

ecg_output.rename(columns={0:'ECG Results'},inplace = True)
class_mapping = {0:'Normal ECG', 1: 'Abnormal ECG' }
ecg_output['ECG Results'].replace(class_mapping, inplace=True)

In [36]:
ecg_output

,index,ECG Results
0,0,Normal ECG
1,1,Abnormal ECG
2,2,Abnormal ECG
3,3,Normal ECG
4,4,Abnormal ECG
...,...,...
166,166,Abnormal ECG
167,167,Abnormal ECG
168,168,Normal ECG
169,169,Abnormal ECG


In [ ]:
# Make predictions using the loaded model and obtain prediction probabilities
ecg_pred = pd.merge(ecg_df_merge,ecg_output,on='index',how ='inner')
ecg_pred.drop(columns = 'index',inplace = True)
ecg_pred.rename(columns={0:'Alive',1:'Dead'},inplace = True)
ecg_pred

In [58]:
ecg_pred

NameError: name 'ecg_pred' is not defined

In [57]:

# Load path for SQL query file
sql_file_path = r'C:/Users/krishika.R/Desktop/Streamlit/app_draft/queries/ed_query.sql'
with open(sql_file_path, 'r') as sql_file:
    sql_queries = sql_file.read().split(';')
    result_dataframes = []
# Replace placeholders with formatted dates
sql_query = sql_queries[2]
print(sql_query)





----------------Query 3: ECG----------------
select * from 
(
select 
distinct a. subject_id,
a.gender,
a.anchor_age,
a.dod,
to_date(REPLACE(b.edadmittime,'2130','2023'),'YYYY-MM-DD') edadmit_time,
to_date(REPLACE(a.ecgtime,'2130','2023'),'YYYY-MM-DD') ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_onset,
a.p_end,
a.qrs_onset,
a.qrs_end,
a.t_end,
a.p_axis,
a.qrs_axis,
a.t_axis,
a.target_variable
from 
factihealth.mimic.ecg_model_dataset a
inner join 
factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where date_part(year,b.edadmittime) = 2130
and date_part(year,a.ecgtime) = 2130
and ecgtime between b.edadmittime and b.eddischargetime
order by ecgtime,a.subject_id
)

----------------Query 4: ECG----------------
select * from 
(
select 
distinct a. subject_id,
a.gender,
a.anchor_age,
to_date(REPLACE(b.edadmittime,'2130','2023'),'YYYY-MM-DD') edadmit_time,
to_date(REPLACE(a.ecgtime,'2130','2023'),'YYYY-MM-DD') ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_o

In [ ]:
try:
        cur.execute(sql_query)
        result = cursor.fetchall()
        df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
        result_dataframes.append(df)
        print("Query executed successfully.")
        return result_dataframes[0] if result_dataframes else None
    except Exception as e:
        print(f"Error executing the query: {str(e)}")
        cur.close()
        conn.close()
        return None

In [51]:
data=ecg_features()
data

Error executing the query: syntax error at or near "select"
LINE 36: select * from 
         ^



In [60]:
data

In [46]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database

conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
print("Connected to the database successfully")
# Create a cursor object
cur = conn.cursor()

Connected to the database successfully


In [ ]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute('''
    select * from 
(
select 
distinct a. subject_id,
a.gender,
a.anchor_age,
a.dod,
to_date(REPLACE(a.ecgtime,'2130','2023'),'YYYY-MM-DD') ecg_time,
a.bandwidth,
a.filtering,
a.rr_interval,
a.p_onset,
a.p_end,
a.qrs_onset,
a.qrs_end,
a.t_end,
a.p_axis,
a.qrs_axis,
a.t_axis,
a.target_variable
from 
factihealth.mimic.ecg_model_dataset a
inner join 
factihealth.mimic.mortality_data b on a.subject_id=b.subject_id
where date_part(year,b.edadmittime) = 2130
and date_part(year,a.ecgtime) = 2130
and ecgtime between b.edadmittime and b.eddischargetime
order by ecgtime,a.subject_id
)
--where ecg_time between '2023-01-01' AND '2023-01-31'
    ''')
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)
    

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")